In [67]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import re

In [68]:
import re

def extract_clean_dialogues(filepath, max_lines=20000):
    dialogues = []
    with open(filepath, encoding='utf-8', errors='ignore') as f:
        for idx, line in enumerate(f):
            if idx >= max_lines:
                break
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                text = parts[4].lower()
                text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # remove punctuation
                tokens = text.split()
                if len(tokens) > 1:
                    dialogues.append(tokens)
    return dialogues

In [69]:
file_path = "movie_lines.txt"
dialogues = extract_clean_dialogues(file_path)
print(f"Loaded {len(dialogues)} cleaned dialogues")
print("Sample:", dialogues[:5])

Loaded 18359 cleaned dialogues
Sample: [['they', 'do', 'not'], ['they', 'do', 'to'], ['i', 'hope', 'so'], ['she', 'okay'], ['lets', 'go']]


In [70]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def prepare_sequences(token_lists, seq_length=5):
    # Flatten the list of tokens
    all_tokens = [word for line in token_lists for word in line]
    # ['i', 'am', 'happy', 'you', 'are', 'great']

    # Create vocabulary
    vocab = sorted(set(all_tokens))
    # ['am', 'are', 'great', 'happy', 'i', 'you']

    word2idx = {word: idx+1 for idx, word in enumerate(vocab)}  # reserve 0 for padding
    idx2word = {idx: word for word, idx in word2idx.items()}

    #word2idx = {'am':1, 'are':2, 'great':3, 'happy':4, 'i':5, 'you':6}
    #idx2word = {1:'am', 2:'are', 3:'great', 4:'happy', 5:'i', 6:'you'}

    # Generate sequences
    sequences = []
    for i in range(seq_length, len(all_tokens)):
        seq = all_tokens[i-seq_length:i+1]  # n words + 1 target
        sequences.append([word2idx[word] for word in seq])

   #i = 3 → ['i', 'am', 'happy', 'you'] → X: ['i', 'am', 'happy'], y: 'you'
   #i = 4 → ['am', 'happy', 'you', 'are'] → X: ['am', 'happy', 'you'], y: 'are'
   #i = 5 → ['happy', 'you', 'are', 'great'] → X: ['happy', 'you', 'are'], y: 'great'

    sequences = np.array(sequences)
    X = sequences[:, :-1]
    y = sequences[:, -1]

    '''sequences = [
    [5, 1, 4, 6],   # 'i', 'am', 'happy', 'you'
    [1, 4, 6, 2],   # 'am', 'happy', 'you', 'are'
    [4, 6, 2, 3]    # 'happy', 'you', 'are', 'great'
]
Then:

    X = sequences[:, :-1]  # all except last token → input
    y = sequences[:, -1]   # last token → target

    # X = [
    #   [5, 1, 4],  # i am happy
    #   [1, 4, 6],  # am happy you
    #   [4, 6, 2]   # happy you are
    # ]

    # y = [6, 2, 3]  # you, are, great'''
    return X, y, word2idx, idx2word

In [72]:
import pickle

# Save word2idx
with open("word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)

# Save idx2word
with open("idx2word.pkl", "wb") as f:
    pickle.dump(idx2word, f)

In [29]:
X, y, word2idx, idx2word = prepare_sequences(dialogues, seq_length=5)

print("Input sequence (X[0]):", X[0])
print("Target word index (y[0]):", y[0])
print("Target word (decoded):", idx2word[y[0]])
print("Vocabulary size:", len(word2idx))

Input sequence (X[0]): [11425  3298  7693 11425  3298]
Target word index (y[0]): 11563
Target word (decoded): to
Vocabulary size: 12916


In [30]:
vocab_size = len(word2idx) + 1  # +1 for padding (index 0)
seq_length = X.shape[1]

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))

In [32]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
with tf.device('/GPU:0'):
    model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
1571/1571 [==============================] - 35s 22ms/step - loss: 6.6589 - accuracy: 0.0437
Epoch 2/100
1571/1571 [==============================] - 30s 19ms/step - loss: 6.0934 - accuracy: 0.0725
Epoch 3/100
1571/1571 [==============================] - 34s 22ms/step - loss: 5.7592 - accuracy: 0.1014
Epoch 4/100
1571/1571 [==============================] - 33s 21ms/step - loss: 5.5156 - accuracy: 0.1153
Epoch 5/100
1571/1571 [==============================] - 35s 22ms/step - loss: 5.3029 - accuracy: 0.1251
Epoch 6/100
1571/1571 [==============================] - 35s 23ms/step - loss: 5.1012 - accuracy: 0.1341
Epoch 7/100
1571/1571 [==============================] - 40s 25ms/step - loss: 4.9074 - accuracy: 0.1437
Epoch 8/100
1571/1571 [==============================] - 30s 19ms/step - loss: 4.7230 - accuracy: 0.1559
Epoch 9/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.5480 - accuracy: 0.1693
Epoch 10/100
1571/1571 [==============================]

In [15]:
model.save("next_word_lstm_model2.h5")

In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def predict_next_word(model, input_words, word2idx, idx2word, seq_length):
    # Convert words to their corresponding indices
    input_seq = [word2idx.get(w.lower(), 0) for w in input_words]

    # Pad the sequence to the required input length
    input_seq = pad_sequences([input_seq], maxlen=seq_length)

    # Predict next word
    pred = model.predict(input_seq, verbose=0)
    pred_idx = np.argmax(pred, axis=-1)[0]

    # Return the predicted word
    return idx2word.get(pred_idx, "<unk>")

In [52]:
predict_next_word(model, ["how", "are", "you"], word2idx, idx2word, 3)

'doing'

In [66]:
predict_next_word(model, ["i", "love", "you"], word2idx, idx2word, 3)

'promise'

In [38]:
predict_next_word(model, ["where", "is", "she"], word2idx, idx2word, 3)

'mistake'

In [39]:
predict_next_word(model, ["what", "is", "your"], word2idx, idx2word, 3)

'fault'

In [40]:
predict_next_word(model, ["do", "you", "want"], word2idx, idx2word, 3)

'to'

In [24]:
predict_next_word(model, ["i", "want", "to", "go", "with"], word2idx, idx2word, seq_length=5)

'you'

In [28]:
predict_next_word(model, ["can", "you", "help", "me", "with"], word2idx, idx2word, seq_length=5)

'my'

In [32]:
predict_next_word(model, ["do", "you", "know", "what", "is"], word2idx, idx2word, seq_length=5)

'it'

In [33]:
predict_next_word(model, ["let", "us", "meet", "at", "the"], word2idx, idx2word, seq_length=5)

'sight'

In [42]:
predict_next_word(model, ["i", "will", "see"], word2idx, idx2word, seq_length=3)

'you'

In [43]:
predict_next_word(model, ["how", "much", "does"], word2idx, idx2word, seq_length=3)

'that'

In [38]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load model
model = load_model('next_word_lstm_model2.h5')


def predict_next_words(seed_text, n_words, seq_len=5):
    result = seed_text.split()
    
    for _ in range(n_words):
        # Get last 'seq_len' words
        input_seq = result[-seq_len:]
        
        # Convert to integer sequence
        token_seq = [word2idx.get(w, 0) for w in input_seq]
        token_seq = pad_sequences([token_seq], maxlen=seq_len)
        
        # Predict next word
        predicted_idx = np.argmax(model.predict(token_seq, verbose=0), axis=-1)[0]
        predicted_word = idx2word.get(predicted_idx, '')
        
        result.append(predicted_word)
    
    return ' '.join(result)
with tf.device('/GPU:0'):
    seed = "how are you doing today"
    generated_text = predict_next_words(seed, n_words=500)
    print("Generated:", generated_text)

Generated: how are you doing today fuck you you cant walk away in the world did you ever turn around with me happy any good father dont make me laugh that was my fault hes in my way oh jesus why bother we are now are already serious you have to do this i brung you a little basket of goodies fresh squash tomatoes some okra butter beans dont make it less true i already tossed that out somethin you shoulda done to me its okay lets go to fight in the end of families ill catch you up ill see him when we get there nobody knows anything about that one if you were on your way it happened with the confession i know its thinking of the first rate and we share with their money you know you didnt come to me nothing matt ill invite him some name yes sir i had to turn on to her this room am in his funeral used and look at debbies inside mind thats what they do oh yes yes i know you found a note in the victims no last right away and this time is right sir at the base office the adventure ive been wor